# Robot stuff

In this notebook we can check how to sample the trajectory of the robot, how to get velocity profiles from the trajectory, which can be used to control the robot, etc.  

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

### Trajectory sampled uniformly in time

In [ ]:
from trajectory import Trajectory
from environment import Environment
from global_variables import EPSILON
from measurements import create_mask, get_D_topright
from solvers import alternativePseudoInverse, exactSolution

n_anchors = 4 #number of anchors
n_complexity = 3 #complexity of trajectory
dim = 2 # dimension of setup. 
n_positions = 10
model = 'full_bandlimited'
# model = 'polynomial'

trajectory = Trajectory(n_complexity, dim=dim, model=model, full_period=True)
# trajectory.set_coeffs(seed=1)
trajectory.coeffs = np.zeros_like(trajectory.coeffs)
trajectory.coeffs[0, 1] = 1
trajectory.coeffs[1, 2] = 1

times_nonunif = trajectory.get_times(n_samples=n_positions)
basis_nonunif = trajectory.get_basis(times=times_nonunif + 1e-10)
sample_points_nonunif = trajectory.get_sampling_points(basis=basis_nonunif)

plt.figure()
trajectory.plot(basis_nonunif, color='orange', label='ground truth')
plt.legend()

### Scale the trajectory to a given bounding box

In [ ]:
from matplotlib.patches import Rectangle

box = np.array([2.0, 2.0])
box = trajectory.scale_bounding_box(box, keep_aspect_ratio=False)
fig, ax = plt.subplots(1)
trajectory.plot(basis_nonunif)
p = Rectangle((0, 0), box[0], box[1], linewidth=1, edgecolor='c',facecolor='none')
ax.add_patch(p)
plt.show()

### Trajectory sampled uniformly in distance

In [ ]:
from trajectory import Trajectory
from environment import Environment

from measurements import create_mask, get_D_topright
from solvers import alternativePseudoInverse, exactSolution

times_unif, distances, errors = trajectory.get_times_unform_in_path(
    n_samples=n_positions,
#     step_distance=0.5,
    time_steps=50000,
    plot=True)
print("cumulative error: {}".format(np.linalg.norm(errors)))
basis_unif = trajectory.get_basis(times=times_unif)
sample_points_unif = trajectory.get_sampling_points(basis=basis_unif)

plt.figure()
# environment.plot()
trajectory.plot(basis_nonunif, color='blue', label='non uniform')
trajectory.plot(basis_unif, color='orange', marker='x', label='uniform')
plt.legend()

## Compute the curvature at each point. 

For now there is still a problem. Since the points are not resampled in time, there is not only a radial acceleration (the one that makes the robot turn) but also a tangential radiation. In plot below, the orange lines showing the curvature radius, are therefore oriented in the direction of the curve. 

For now I am not quite sure if this is a problem, or if we can still try to incorporate this in the controls for the robot. 

In [ ]:
fig, ax = plt.subplots()
trajectory.plot(basis_nonunif, color='orange', label='ground truth')
xlims = ax.get_xlim()
ylims = ax.get_ylim()
plt.axis('equal')

radii, unit_tangents, radial_acc = trajectory.get_curvatures(times_nonunif, ax=ax)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
#plt.xlim([-2, 8])
#plt.ylim([-2, 5])

In [ ]:
# TODO(FD) Below needs some refactoring, 
# there is a lot of duplicate code. 

fig, ax = plt.subplots()
trajectory.plot(basis_unif)
plt.axis('equal')

ROBOT_WIDTH = 0.1
points_left, points_right = trajectory.get_left_and_right_points(times_unif, width=ROBOT_WIDTH, ax=ax)

## Generate trajectories for both wheels 

In [ ]:
import csv

FOLDER = 'controls/'

def generate_control_parameters(seed,
                                n_complexity,
                                model,
                                n_samples=None,
                                step_distance=None,
                                fname='',
                                box=np.array([2.0, 2.0])):
    width = 0.3 # width of the robot
    
    
    # This is a hack, see issue #22: https://github.com/duembgen/SamplingTrajectories/issues/22
    if model == 'circle':
        trajectory = Trajectory(n_complexity=3, model='full_bandlimited') 
        trajectory.coeffs = np.zeros_like(trajectory.coeffs)
        trajectory.coeffs[0, 1] = 1
        trajectory.coeffs[1, 2] = 1
    elif model == 'eight':
        trajectory = Trajectory(n_complexity=5, model='full_bandlimited') 
        trajectory.coeffs = np.zeros_like(trajectory.coeffs)
        trajectory.coeffs[0, 1] = 1
        trajectory.coeffs[1, 3] = 1
    else:
        trajectory = Trajectory(n_complexity=n_complexity, model=model) 
        trajectory.set_coeffs(seed=seed, dimension=2)
    
    trajectory.scale_bounding_box(box)
    times, distances, errors = trajectory.get_times_unform_in_path(
        n_samples=n_samples,
        step_distance=step_distance,
        plot=False)
    # at time zero all the derivatives are zero, so it's 
    times = times + EPSILON
    
    basis = trajectory.get_basis(times=times)
    sample_points = trajectory.get_sampling_points(basis=basis)
    
    fig, ax = plt.subplots()
    trajectory.plot(basis, color='orange', label='trajectory')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    xlims = ax.get_xlim()
    ylims = ax.get_ylim()
    plt.axis('equal')
    
    radii, __, __ = trajectory.get_curvatures(times=times, ax=None)
    pos_left, pos_right = trajectory.get_left_and_right_points(times=times, width=width, ax=ax)
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    ax.legend()
    plt.savefig(FOLDER + fname + '.png')
    print('wrote plots as', FOLDER + fname + '.png')
    
    if fname[-4:] != '.txt':
        fnametxt = fname + '.txt'
    else:
        fnametxt = fname
        fname = fnametxt[:-4]
    with open(FOLDER + fnametxt, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        header = ['i', 'rad', 'dist', 'x', 'y', 'x_left', 'y_left', 'x_right', 'y_right']
        #print(header)
        writer.writerow(header)
        for i in range(len(distances) - 1):
            r = radii[i + 1]
            delta_d = distances[i + 1]  - distances[i]
            p = sample_points[:, i]
            pl = pos_left[:, i]
            pr = pos_right[:, i]
            line = [i, r, delta_d, p[0], p[1], pl[0], pl[1], pr[0], pr[1]]
            #print(np.array(line))
            writer.writerow(line)
        print('wrote parameters as', FOLDER + fnametxt)

trajectories = [
    dict(
        fname = 'bandlimited1',
        model = 'bandlimited',
        n_complexity = 3,
        step_distance = 0.1,
        seed = 2
    ), 
    dict(
        fname = 'polynomial1',
        model = 'polynomial',
        n_complexity = 3,
        step_distance = 0.1,
        seed = 1
    ), 
    dict(
        fname = 'full_bandlimited1',
        model = 'full_bandlimited',
        n_complexity = 5,
        step_distance = 0.1,
        seed = 3
    ), 
    dict(
        fname = 'circle1',
        model = 'circle',
        n_complexity = 5,
        step_distance = 0.1,
        seed = 0
    ), 
    dict(
        fname = 'eight1',
        model = 'eight',
        n_complexity = 5,
        step_distance = 0.1,
        seed = 0
    ), 
]

for traj in trajectories:
    generate_control_parameters(**traj)